In [1]:
import pandas as pd
train = pd.read_csv("data-transformed/05-for-bayes-sample.csv")

In [4]:
train.describe()

,Id,L0_S0_,L0_S1_,L0_S10,L0_S11,L0_S12,L0_S13,L0_S14,L0_S15,L0_S16,...,L3_S43,L3_S44,L3_S45,L3_S46,L3_S47,L3_S48,L3_S49,L3_S50,L3_S51,Response
count,71025.000000,71025.000000,71025.000000,71025.000000,71025.000000,71025.000000,71025.000000,71025.000000,71025.000000,71025.000000,...,71025.000000,71025.000000,71025.000000,71025,71025.000000,71025.000000,71025.000000,71025.000000,71025.000000,71025.000000
mean,1184742.989849,0.565618,0.565660,0.188187,0.189046,0.205660,0.205660,0.102330,0.103344,0.100641,...,0.025160,0.024569,0.049321,0,0.049293,0.049377,0.023977,0.025456,0.049349,0.006040
std,683149.652733,0.495679,0.495674,0.390864,0.391548,0.404186,0.404186,0.303084,0.304410,0.300854,...,0.156612,0.154808,0.216538,0,0.216480,0.216655,0.152980,0.157506,0.216597,0.077484
min,16.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,596379.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1186834.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1772914.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2367440.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [2]:
response = train.Response
del train['Id']
del train['Response']

In [6]:
response.describe()

count    71025.000000
mean         0.006040
std          0.077484
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: Response, dtype: float64

### Niave Bayes Bernoulli

In [5]:
from sklearn.metrics import make_scorer
from sklearn.metrics import matthews_corrcoef
from sklearn import grid_search
from sklearn.naive_bayes import BernoulliNB

matt_score = make_scorer(matthews_corrcoef)

param_grid = {'alpha': [0, 0.25, 0.5, 0.75, 1]}

nb = BernoulliNB()

clf = grid_search.GridSearchCV(nb, param_grid, cv = 3, n_jobs = 3, scoring = matt_score)

clf.fit(train, response)
for params, mean_score, scores in clf.grid_scores_:
        print("%0.3f (+/-%0.03f) for %r"
              % (mean_score, scores.std() * 2, params))

0.000 (+/-0.000) for {'alpha': 0}
0.063 (+/-0.049) for {'alpha': 0.25}
0.070 (+/-0.027) for {'alpha': 0.5}
0.071 (+/-0.028) for {'alpha': 0.75}
0.072 (+/-0.030) for {'alpha': 1}


### Linear SVM

In [16]:
from sklearn.svm import LinearSVC
matt_score = make_scorer(matthews_corrcoef)

param_grid = {'C': [0.1, 1, 10]
             }

svm = LinearSVC(dual = False, penalty = "l1", loss = 'squared_hinge', class_weight = 'auto') 
#Docs say make it dual false when n_samples > n_features
#Penality and loss have to be l1 and squared_hinge when using dual-False
#Also tried dual = True but no better

clf = grid_search.GridSearchCV(svm, param_grid, cv = 3, n_jobs = 3, scoring = matt_score)

clf.fit(train, response)
for params, mean_score, scores in clf.grid_scores_:
        print("%0.3f (+/-%0.03f) for %r"
              % (mean_score, scores.std() * 2, params))

0.026 (+/-0.031) for {'C': 0.1}
0.018 (+/-0.021) for {'C': 1}
0.018 (+/-0.018) for {'C': 10}


### Random Forest

In [20]:
from sklearn.ensemble import RandomForestClassifier
matt_score = make_scorer(matthews_corrcoef)

param_grid = {'n_estimators': [500],
              'criterion': ['gini', 'entropy'],
              'min_samples_split': [2, 3, 4]
             }

rf = RandomForestClassifier() 
#Docs say make it dual false when n_samples > n_features
#Penality and loss have to be l1 and squared_hinge when using dual-False
#Also tried dual = True but no better

clf = grid_search.GridSearchCV(rf, param_grid, cv = 3, n_jobs = 3, scoring = matt_score)

clf.fit(train, response)
for params, mean_score, scores in clf.grid_scores_:
        print("%0.3f (+/-%0.03f) for %r"
              % (mean_score, scores.std() * 2, params))


0.078 (+/-0.054) for {'min_samples_split': 2, 'criterion': 'gini', 'n_estimators': 500}
0.058 (+/-0.041) for {'min_samples_split': 3, 'criterion': 'gini', 'n_estimators': 500}
0.028 (+/-0.052) for {'min_samples_split': 4, 'n_estimators': 500, 'criterion': 'gini'}
0.078 (+/-0.054) for {'min_samples_split': 2, 'n_estimators': 500, 'criterion': 'entropy'}
0.050 (+/-0.023) for {'min_samples_split': 3, 'n_estimators': 500, 'criterion': 'entropy'}
0.026 (+/-0.048) for {'min_samples_split': 4, 'n_estimators': 500, 'criterion': 'entropy'}


Random Forest performs well. Just add these features to the main analysis vector

### RandomForest feature importance

In [29]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators = 1000)
rf.fit(train, response)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [28]:
importances = pd.DataFrame({'feature': train.columns.values,
                           'importances': rf.feature_importances_})
importances.sort('importances', ascending = False)

,feature,importances
34,L3_S34,0.091309
33,L3_S33,0.071996
38,L3_S38,0.068843
36,L3_S36,0.064954
26,L2_S26,0.059957
35,L3_S35,0.055485
32,L3_S32,0.046589
27,L2_S27,0.046206
17,L0_S3_,0.035406
12,L0_S2_,0.034093
